<a id="toc"></a>
# Table of Contents

1.) [Intro](#intro)  
2.) [Imports](#imports)  
3.) [Setup](#setup)  
4.) [Helpers](#helpers)  

<a id="intro"></a>
### [^](#toc) Intro

In the original SlowMo [paper presented here](https://people.cs.umass.edu/~hzjiang//projects/superslomo/) [and here](https://arxiv.org/abs/1712.00080), the team trained on 1,132 Youtube videos or 300K individual frames.

While this sounds like a lot we have just as much.  CONNECTOME, ENGAGE, and RAD have a combined ~1,500 sessions on Flywheel and a estimate of 3 tasks per session and 150 volumes per task mean we have 675,000 individual frames

In [1]:
mask_num = 20

"Number of features:", (
    (247 * 2) + # 4 radius voxels (previous and next) (not directly adjacent)
    (2 * mask_num * 2) + # (loop over masks) mean/std of mask (next and previous)
    2 + # time since (until) previous (next)
    1 + # time ratio between next voxels and previous
    2 + # time since last gonogo stimuli
    2 + # time since go/nogo bloack started
    6 + # time since faces stimuli
    6 + # time since faces block started
    1 + # time since keypress (any)
    3 + # gender, age, scl20
    4 + # task identifier (is_gonogo, is_(non)conscious, is_wm)
    3 + # study identifier (conn, engage, rad)
    mask_num + # flags if voxel within mask
    1 + # is_grey_matter
    0
)

('Number of features:', 625)

<a id="imports"></a>
### [^](#toc) Imports

In [7]:
import os
import glob

import numpy as np
import pandas as pd
import nibabel as nib

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<a id="setup"></a>
### [^](#toc) Setup

In [8]:
masks_path = "/Volumes/group/pstetz/git/PANLab/PANLab_fMRI_scripts/config/PanLab_Masks_Biotypes_May2019.csv"
masks_dir  = "/Volumes/group/PANLab_Datasets/Masks/Parcels/Neurosynth_maps/Nature_Medicine_RR/ROI_masks_for_revision"

with open(masks_path) as f:
    mask_names = f.readlines()
    mask_names = [m[:-1] for m in mask_names] # remove \n character
    
masks = [os.path.join(masks_dir, m + ".nii") for m in mask_names]

<a id="helpers"></a>
### [^](#toc) Helpers

In [ ]:
def get_data(filepath):
    assert os.path.isfile(filepath), "%s does not exist"
    image = nib.load(fn)
    return nib.as_closest_canonical(image)

def mask_image(image, mask):
    if len(image.shape) == 3:
        return np.multiply(image, mask)
    assert len(image.shape) == 4, "%s is not a valid image shape." % str(image.shape)
    
    masked_imaged = image.copy()
    for i in range(image.shape[3]):
        masked_image[i] = np.multiply(image[i], mask)
    return masked_image

def mask_mean_std(image, mask):
    """ FIXME: might want to norm each voxel before taking the mean/std """
    masked_image = mask_image(image, mask)
    return mask_image.mean(), mask_image.std()

def norm_voxel(data, x, y, z, t):
    assert len(data.shape) == 4, "The dimension size should be 4 not %s" % str(data.shape)
    voxel = data[x, y, z, :]
    return (voxel[t] - voxel.mean()) / voxel.std()

def trainable_mask(mask, radius):
    """ Given a mask determine what voxels are [radius] away from the outside """
    trainable_mask = mask.copy()
    x_size, y_size, z_size = mask.shape
    for x in range(x_size):
        for y in range(y_size):
            for z in range(z_size):
                if mask[x, y, z] == 0:
                    continue
                if not within_radius(mask, x, y, z, radius):
                    trainable_mask[x, y, z] = 0
    return trainable_mask

def voxel_radius(radius):
    valid = list()
    count = 0

    for i in range(radius+1):
        for j in range(radius+1):
            for k in range(radius+1):
                if i == 0 and j == 0 and k == 0:
                    continue
                if i**2 + j**2 + k**2 > radius**2:
                    continue

                for parity in ([1, 1, 1], [-1, 1, 1], [1, -1, 1], [1, 1, -1],
                               [-1, -1, 1], [-1, 1, -1], [1, -1, -1], [-1, -1, -1]):
                    if ((i == 0 and parity[0] == -1) or
                        (j == 0 and parity[1] == -1) or
                        (k == 0 and parity[2] == -1)):
                        continue
                    valid.append({"x":  i * parity[0], "y":  j * parity[1], "z":  k * parity[2]})
    print(len(valid), (2 * radius + 1)**3)
    return valid
#     return count, (2 * radius + 1)**3

In [42]:
voxel_radius(4);

256 729
